# Oversampling, W-Projection, WENSS setup

A tweakable end-to-end runthrough using WENSS setup. Tests W-Projection with oversampling.

In [ ]:
import numpy as np
import logging
import os

import astropy.units as u
import astropy.constants as const
import fastimgproto.imager as imager
from fastimgproto.gridder.gridder import convolve_to_grid
import fastimgproto.visibility as visibility

from astropy.coordinates import Angle, SkyCoord, AltAz, EarthLocation
from astropy.time import Time
from fastimgproto.gridder.conv_funcs import GaussianSinc
from fastimgproto.skymodel.helpers import SkyRegion, SkySource
from fastimgproto.sourcefind.image import SourceFindImage
from fastimgproto.telescope.readymade import VLA_C

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# Plot image pixels in cartesian ordering (i.e. y-positive == upwards):
plt.rcParams['image.origin'] = 'lower'
# Make plots bigger
plt.rcParams['figure.figsize'] = 10, 10

## Set up the telescope

In [ ]:
telescope = VLA_C()

In [ ]:
pointing_centre = SkyCoord(194.24 * u.deg, 47.339 * u.deg)
obs_central_frequency = 74 * u.MHz
wavelength = const.c / obs_central_frequency
transit_time = telescope.next_transit(pointing_centre.ra,
                                      start_time=Time('2017-01-01'))

Choose the observation times / sampling regime

In [ ]:
nstep=1440
obs_times = transit_time + np.linspace(-6, 6, nstep) * u.hr
print("Generating UVW-baselines for {} timesteps".format(nstep))
uvw_m = telescope.uvw_tracking_skycoord(pointing_centre, obs_times)
# From here on we use UVW as multiples of wavelength, lambda:
uvw_lambda = (uvw_m / wavelength).to(u.dimensionless_unscaled).value

## Configure the sources to simulate

WENSS sources list

In [ ]:
sources_data=np.matrix([[ 1, 11, 47, 52.21, 48, 18, 49.2, 2033], [ 2, 11, 51, 9.26, 47, 28, 55.7, 2889], [ 3, 11, 53, 24.60, 49, 31, 9.6, 4041], [ 4, 11, 53, 46.43, 51, 17, 4.6, 2328], [ 5, 11, 54, 20.56, 45, 23, 29.7, 3012], [ 6, 11, 56, 9.31, 44, 50, 20.0, 2223], [ 7, 11, 59, 13.55, 53, 53, 7.8, 4673], [ 8, 11, 59, 13.83, 53, 53, 7.1, 4595], [ 9, 12, 0, 31.11, 45, 48, 42.3, 3170], [10, 12, 7, 14.14, 54, 7, 54.0, 2677], [11, 12, 9, 13.62, 43, 39, 18.5, 7220], [12, 12, 13, 29.92, 48, 23, 12.9, 2021], [13, 12, 15, 29.88, 53, 35, 50.2, 7827], [14, 12, 22, 51.88, 50, 26, 55.0, 2570], [15, 12, 24, 28.60, 42, 6, 34.5, 5127], [16, 12, 24, 29.19, 42, 6, 49.7, 3565], [17, 12, 30, 34.50, 41, 38, 58.1, 2081], [18, 12, 34, 29.72, 41, 9, 38.6, 2239], [19, 12, 36, 50.00, 36, 55, 18.0, 3277], [20, 12, 41, 56.97, 57, 30, 44.6, 4347], [21, 12, 44, 49.15, 40, 48, 6.5, 2139], [22, 12, 44, 49.46, 36, 9, 23.0, 2803], [23, 12, 46, 38.99, 56, 49, 21.1, 2472], [24, 12, 46, 46.05, 38, 41, 40.0, 2653], [25, 12, 47, 7.40, 49, 0, 19.5, 2430], [26, 12, 49, 23.07, 44, 44, 46.5, 2240], [27, 12, 51, 42.70, 50, 34, 22.8, 4672], [28, 12, 51, 46.60, 50, 34, 34.1, 2960], [29, 12, 52, 8.60, 52, 45, 30.5, 4610], [30, 12, 52, 16.79, 47, 15, 36.4, 4378], [31, 12, 52, 26.33, 56, 34, 20.6, 6792], [32, 12, 56, 17.84, 37, 13, 42.8, 2917], [33, 12, 56, 57.42, 47, 20, 20.8, 16888], [34, 12, 57, 23.56, 36, 44, 26.4, 2438], [35, 12, 57, 23.88, 36, 44, 19.1, 2312], [36, 12, 58, 1.93, 44, 35, 21.7, 3703], [37, 13, 0, 13.75, 38, 4, 32.9, 2654], [38, 13, 0, 32.93, 40, 9, 7.9, 5688], [39, 13, 3, 43.99, 37, 56, 9.9, 2046], [40, 13, 4, 28.85, 53, 50, 0.9, 3118], [41, 13, 4, 57.84, 38, 32, 29.8, 2462], [42, 13, 16, 12.83, 45, 4, 38.7, 2245], [43, 13, 19, 46.03, 51, 48, 11.7, 3071], [44, 13, 21, 13.87, 42, 34, 42.2, 2858], [45, 13, 21, 18.07, 42, 35, 0.6, 6188], [46, 13, 21, 21.65, 42, 35, 16.3, 3329], [47, 13, 23, 24.08, 41, 15, 12.9, 2293], [48, 13, 23, 24.29, 41, 15, 15.1, 2218], [49, 13, 26, 2.25, 36, 47, 59.3, 2596], [50, 13, 26, 13.98, 49, 34, 34.4, 2231], [51, 13, 27, 37.17, 55, 4, 6.2, 2926], [52, 13, 29, 54.91, 47, 12, 26.8, 3789], [53, 13, 31, 37.37, 50, 7, 57.4, 3505], [54, 13, 31, 37.38, 50, 7, 57.6, 3478], [55, 13, 35, 19.45, 41, 0, 2.4, 3836], [56, 13, 37, 38.58, 47, 41, 47.7, 2036], [57, 13, 37, 38.65, 47, 41, 49.0, 2070], [58, 13, 38, 49.36, 38, 51, 14.6, 12507], [59, 13, 38, 49.37, 38, 51, 14.5, 12494], [60, 13, 41, 33.60, 53, 44, 50.1, 3607], [61, 13, 41, 44.87, 46, 57, 17.2, 2677], [62, 13, 45, 26.34, 49, 46, 32.7, 8796], [63, 13, 45, 33.08, 42, 50, 14.7, 3242], [64, 13, 46, 23.66, 48, 13, 10.0, 2034], [65, 13, 49, 34.67, 53, 41, 17.4, 2186], [66, 14, 0, 19.00, 53, 36, 59.8, 2900]])

all_sources = []
for idx in range(0, sources_data.shape[0]):
    positions = SkyCoord(ra=(sources_data[idx,1]+sources_data[idx,2]/60+sources_data[idx,3]/3600)*15*u.deg, dec=((sources_data[idx,4]+sources_data[idx,5]/60+sources_data[idx,6]/3600)*u.deg))
    source=SkySource(position=positions, flux=sources_data[idx,7] * 2.83 * u.mJy)
    all_sources.append(source)
all_sources

## Simulate visibilities, given the source-list and UVW baseline info:

In [ ]:
# Simulate incoming data; includes transient sources, noise:
#baseline_noise_level = 0.001 * u.Jy

data_vis = visibility.visibilities_for_source_list(
    pointing_centre,
    source_list = all_sources, 
    uvw = uvw_lambda)

#data_vis = visibility.add_gaussian_noise(baseline_noise_level, data_vis)

snr_weights = np.ones_like(data_vis,dtype=np.float_)

# Invert v signal
uvw_lambda[:,1]*=-1

## Configure and run the imager

In [ ]:
image_size = 768 * u.pixel
cell_size = 60 * u.arcsecond

In [ ]:
kernel_support = 3
kernel_func = GaussianSinc(trunc=kernel_support)

## Pipeline

Run pipeline using several oversampling values

In [ ]:
sfimage_list = []
oversampling_list = [1, 2, 4, 6, 8, 10, 12, 14, 16]

for oversampling in oversampling_list:
    image, beam = imager.image_visibilities(
        data_vis,
        snr_weights,
        uvw_lambda,
        image_size=image_size,
        cell_size=cell_size,
        kernel_func=kernel_func,
        kernel_support=kernel_support,
        kernel_exact=False,
        kernel_oversampling=oversampling,
        num_wplanes=32,
        max_wpconv_support=25,
        )
    image = np.real(image)
    beam = np.real(beam)
    
    detection_n_sigma=5
    analysis_n_sigma=5
    sfimage = SourceFindImage(data=np.real(image),
                              detection_n_sigma=detection_n_sigma,
                              analysis_n_sigma=analysis_n_sigma,
                              )
    sfimage_list.append(sfimage)

## Analysis of Results

In [ ]:
rms_list = []
numisl_list = []

for img in sfimage_list:
    rms_list.append(img.rms_est)
    numisl_list.append(np.shape(img.islands))

In [ ]:
rms_list

In [ ]:
numisl_list

### Plot peak-flux and dynamic-range as a function of oversampling

In [ ]:
extremum_list = []
idx = 1
for risl in sfimage_list[0].islands:
    peakflux_list = []
    
    # Not all sources are detected by source find algorithm for all oversampling cases, 
    # so we only plot the sources present in all sfimage objects.
    # The sources need to be searched using the following code:
    for img in sfimage_list:
        for isl in img.islands:
            other_island = []
            if risl.params.extremum.index.x == isl.params.extremum.index.x and risl.params.extremum.index.y == isl.params.extremum.index.y:
                other_island = isl
                break
        
        if not other_island:
            break
            
        peakflux_list.append(other_island.params.extremum.value)
        
    if len(peakflux_list) == len(sfimage_list):
        plt.figure(figsize=(10,4))
        ax1 = plt.subplot(211)
        plt.title('Island %s, Position: (X=%s, Y=%s)' % (idx, np.array_str(risl.params.extremum.index.x),
                                                np.array_str(risl.params.extremum.index.x)))
        plt.plot(oversampling_list, peakflux_list)
        plt.ylabel('Peak flux [Jy]')
        plt.setp(ax1.get_xticklabels(), visible=False)

        ax2 = plt.subplot(212, sharex=ax1)
        plt.plot(oversampling_list, np.array(peakflux_list) / np.array(rms_list))
        plt.ylabel('Dynamic Range')
        plt.xlabel('Oversampling')

        plt.savefig('peakflux_isl_%d.pdf' % idx)
        plt.show()
        idx+=1

### Plot estimated RMS as a function of oversampling 

In [ ]:
plt.figure(figsize=(10,4))
plt.title('RMS Estimation')
plt.plot(oversampling_list, rms_list)
plt.ylabel('RMS estimation')
plt.xlabel('Oversampling')

plt.savefig('rms_estimation.pdf')
plt.show()    